<a href="https://colab.research.google.com/github/BeagleTamer100/256/blob/main/WORKING_ML_Corona_moi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
#update

In [ ]:
!pip install mlforecast

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from mlforecast import MLForecast
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from utilsforecast.evaluation import evaluate
import warnings

warnings.filterwarnings("ignore")

from mlforecast import MLForecast
from mlforecast.feature_engineering import transform_exog
from mlforecast.lag_transforms import RollingMean, ExpandingMean
from typing import List
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Customer Order Quantity_Dispatched Quantity_cleaned.csv')
df.head()

,date_rev,Product ID,Product Name,Date,Customer Order Quantity,Dispatched Quantity,Year,Month,Day,first_week_day
0,2/1/22,000161032,Material_4,01.02.2022,2,4,2022,2,1,1/31/22
1,2/1/23,000161032,Material_4,01.02.2023,60,68,2023,2,1,1/30/23
2,2/1/24,000161032,Material_4,01.02.2024,20,23,2024,2,1,1/29/24
3,3/1/22,000161032,Material_4,01.03.2022,53,0,2022,3,1,2/28/22
4,3/1/23,000161032,Material_4,01.03.2023,99,33,2023,3,1,2/27/23


In [ ]:
weekly_df = df.groupby(['first_week_day', 'Product Name'])['Customer Order Quantity'].sum().reset_index()

In [ ]:
weekly_df.columns = ["ds", "unique_id", "y"]
weekly_df["ds"] = pd.to_datetime(weekly_df["ds"])
weekly_df.head()

,ds,unique_id,y
0,2024-01-01,Material_1,252
1,2024-01-01,Material_10,251
2,2024-01-01,Material_11,120
3,2024-01-01,Material_12,2
4,2024-01-01,Material_13,96


# Features

In [ ]:
# Date features
def _get_date_features(df):
    df["Week"] = df["ds"].dt.isocalendar().week
    df["Week"] = df["Week"].astype(int)
    df["Month"] = df["ds"].dt.month
    df["Year"] = df["ds"].dt.year

    return df

In [ ]:
# Target lag transforms
def _get_lag_transforms(lag_transform_weeks = [2, 4, 6, 8]):
    lag_transforms = []
    for lag in lag_transform_weeks:
        lag_transforms.append(RollingMean(window_size=lag))

    return lag_transforms

In [ ]:
# Target lags
def _get_lags(lags = [1, 2, 4]):
    return lags

In [ ]:
# Add unique_id and sort
weekly_df = weekly_df.sort_values(by=["unique_id", "ds"]).reset_index(drop=True)

In [ ]:
# Add date features
weekly_df = _get_date_features(weekly_df)

In [ ]:
weekly_df.head()

,ds,unique_id,y,Week,Month,Year
0,2022-01-03,Material_1,5,1,1,2022
1,2022-01-10,Material_1,120,2,1,2022
2,2022-01-17,Material_1,60,3,1,2022
3,2022-01-24,Material_1,190,4,1,2022
4,2022-01-31,Material_1,166,5,1,2022


# Model

In [ ]:
# Define a helper function to calculate RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
weekly_df['unique_id'] = weekly_df['unique_id'].astype('category')
train = weekly_df[weekly_df.ds<='2024-06-30']
test = weekly_df[weekly_df.ds>'2024-06-30']

In [ ]:
# Define hyperparameter grids for each model
param_grids = {
    'LGBMRegressor': {
        'objective': ['tweedie'],
        'tweedie_variance_power': [1.3, 1.5, 1.7],
        'learning_rate': [0.05, 0.1],
        'n_estimators': [300, 500],
        'max_depth': [5, 7, 10],
        'random_state': [123]
    },
    'XGBRegressor': {
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 300, 500],
        'max_depth': [5, 7, 10],
        'random_state': [123]
    }
}

In [ ]:
#%% Model inputs
horizon = 13
freq = "W-MON"

In [ ]:
# Prepare train and validation datasets
train_data = train.copy()
valid_data = test.copy()  # Use test data for validation

# Initialize lists to store forecasts and results
forecasts_list = []

# Loop through models and hyperparameter grids
for model_name, param_grid in param_grids.items():
    for params in ParameterGrid(param_grid):
        # Select the appropriate model
        if model_name == 'LGBMRegressor':
            model = LGBMRegressor(**params)
        elif model_name == 'XGBRegressor':
            model = XGBRegressor(**params)

        # Create the MLForecast object with the current model
        fcst = MLForecast(
            models=[model],
            freq=freq,  # Weekly frequency
            lags=_get_lags(),  # Add meaningful lags
            lag_transforms={
                1: _get_lag_transforms(),
            }
        )

        # Train the model on the training data
        fcst.fit(train_data, static_features=[])

        # Generate the required future dataframe
        future_df = fcst.make_future_dataframe(h=horizon)

        # Add date features to the future dataframe
        future_df = _get_date_features(future_df)

        # Generate predictions for the forecast horizon (13 weeks per unique_id)
        forecasts_df = fcst.predict(h=horizon, X_df=future_df)

        # Create a hyperparameter-specific column name
        param_str = f"{model_name}_" + "_".join([f"{k}={v}" for k, v in params.items()])
        forecasts_df.rename(columns={model_name: param_str}, inplace=True)

        # Add the forecasts to the list for merging
        forecasts_list.append(forecasts_df[['unique_id', 'ds', param_str]])

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
# # Merge all model forecasts with the validation data
full_data_forecast = valid_data[['unique_id', 'ds', 'y']]  # Keep only relevant columns
for forecast in forecasts_list:
    full_data_forecast = pd.merge(full_data_forecast, forecast, on=['unique_id', 'ds'], how='left')

In [ ]:
full_data_forecast

,unique_id,ds,y,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.3,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.5,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.7,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=500_objective=tweedie_random_state=123_tweedie_variance_power=1.3,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=500_objective=tweedie_random_state=123_tweedie_variance_power=1.5,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=500_objective=tweedie_random_state=123_tweedie_variance_power=1.7,LGBMRegressor_learning_rate=0.05_max_depth=7_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.3,...,XGBRegressor_learning_rate=0.05_max_depth=10_n_estimators=500_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=5_n_estimators=100_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=5_n_estimators=300_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=5_n_estimators=500_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=7_n_estimators=100_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=7_n_estimators=300_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=7_n_estimators=500_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=10_n_estimators=100_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=10_n_estimators=300_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=10_n_estimators=500_random_state=123
0,Material_1,2024-07-01,179,177.139166,178.847288,186.836418,173.436605,172.156634,191.812822,194.532934,...,166.259583,178.828751,212.887985,207.439850,194.978012,213.977905,210.756119,190.646317,199.524460,198.956451
1,Material_1,2024-07-08,255,196.074686,183.743129,188.379308,204.701735,194.857966,195.608866,187.108873,...,231.030624,169.915802,226.274643,269.196167,191.694946,257.790161,273.411499,227.331421,271.926361,274.821228
2,Material_1,2024-07-15,263,199.782982,189.144036,189.651485,213.116651,198.393544,197.083235,195.839896,...,194.533615,177.781342,189.950790,156.377106,190.777756,149.541733,162.510773,183.972031,241.244568,227.494354
3,Material_1,2024-07-22,122,187.274001,181.115335,181.926806,202.574557,192.572191,194.267841,192.419592,...,154.226395,169.915802,214.424423,243.660339,197.496048,230.667892,233.778702,228.260330,189.950455,184.600372
4,Material_1,2024-07-29,244,210.500777,187.020436,188.361164,212.239716,200.508776,212.691907,213.318164,...,204.603806,171.396759,215.752426,189.004288,213.734421,182.158081,207.777405,229.263840,290.011322,251.632629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,Material_9,2024-10-28,141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705,Material_9,2024-11-04,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1706,Material_9,2024-11-11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1707,Material_9,2024-11-18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def calculate_rmse(test, forecasts_df):
    """
    Calculate RMSE for each model by unique_id.

    Parameters:
        test (pd.DataFrame): DataFrame with actual values ('unique_id', 'ds', 'y').
        forecasts_df (pd.DataFrame): DataFrame with forecasted values ('unique_id', 'ds', model columns).

    Returns:
        pd.DataFrame: DataFrame with RMSE scores for each unique_id and model.
    """
    # Merge the test data with the forecasts data on 'unique_id' and 'ds'
    merged_df = pd.merge(test, forecasts_df, on=['unique_id', 'ds'])

    # Initialize a list to store RMSE scores for each unique_id and model
    rmse_scores = []

    # Loop through each model column (excluding 'unique_id' and 'ds')
    model_columns = [col for col in forecasts_df.columns if col not in ['unique_id', 'ds']]

    for model in model_columns:
        # Group by 'unique_id' and calculate RMSE for each group
        grouped_rmse = merged_df.groupby('unique_id').apply(
            lambda x: np.sqrt(np.mean((x['y'] - x[model])**2))
        )
        # Append scores to the list
        rmse_scores.append(pd.DataFrame({
            'unique_id': grouped_rmse.index,
            'model': model,
            'rmse': grouped_rmse.values
        }))

    # Concatenate the scores into a single DataFrame
    rmse_scores_df = pd.concat(rmse_scores, ignore_index=True)

    return rmse_scores_df

In [ ]:
full_data_forecast.head()

,unique_id,ds,y,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.3,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.5,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.7,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=500_objective=tweedie_random_state=123_tweedie_variance_power=1.3,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=500_objective=tweedie_random_state=123_tweedie_variance_power=1.5,LGBMRegressor_learning_rate=0.05_max_depth=5_n_estimators=500_objective=tweedie_random_state=123_tweedie_variance_power=1.7,LGBMRegressor_learning_rate=0.05_max_depth=7_n_estimators=300_objective=tweedie_random_state=123_tweedie_variance_power=1.3,...,XGBRegressor_learning_rate=0.05_max_depth=10_n_estimators=500_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=5_n_estimators=100_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=5_n_estimators=300_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=5_n_estimators=500_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=7_n_estimators=100_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=7_n_estimators=300_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=7_n_estimators=500_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=10_n_estimators=100_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=10_n_estimators=300_random_state=123,XGBRegressor_learning_rate=0.1_max_depth=10_n_estimators=500_random_state=123
0,Material_1,2024-07-01,179,177.139166,178.847288,186.836418,173.436605,172.156634,191.812822,194.532934,...,166.259583,178.828751,212.887985,207.439850,194.978012,213.977905,210.756119,190.646317,199.524460,198.956451
1,Material_1,2024-07-08,255,196.074686,183.743129,188.379308,204.701735,194.857966,195.608866,187.108873,...,231.030624,169.915802,226.274643,269.196167,191.694946,257.790161,273.411499,227.331421,271.926361,274.821228
2,Material_1,2024-07-15,263,199.782982,189.144036,189.651485,213.116651,198.393544,197.083235,195.839896,...,194.533615,177.781342,189.950790,156.377106,190.777756,149.541733,162.510773,183.972031,241.244568,227.494354
3,Material_1,2024-07-22,122,187.274001,181.115335,181.926806,202.574557,192.572191,194.267841,192.419592,...,154.226395,169.915802,214.424423,243.660339,197.496048,230.667892,233.778702,228.260330,189.950455,184.600372
4,Material_1,2024-07-29,244,210.500777,187.020436,188.361164,212.239716,200.508776,212.691907,213.318164,...,204.603806,171.396759,215.752426,189.004288,213.734421,182.158081,207.777405,229.263840,290.011322,251.632629


In [ ]:
# Remove y from forecasts_df
forecasts_df= full_data_forecast.drop(columns='y')

In [ ]:
evaluation_df = calculate_rmse(valid_data, forecasts_df)
evaluation_df['metric'] = 'rmse'

In [ ]:
evaluation_df

,unique_id,model,rmse,metric
0,Material_1,LGBMRegressor_learning_rate=0.05_max_depth=5_n...,61.890786,rmse
1,Material_10,LGBMRegressor_learning_rate=0.05_max_depth=5_n...,84.161368,rmse
2,Material_11,LGBMRegressor_learning_rate=0.05_max_depth=5_n...,58.390102,rmse
3,Material_12,LGBMRegressor_learning_rate=0.05_max_depth=5_n...,37.996400,rmse
4,Material_13,LGBMRegressor_learning_rate=0.05_max_depth=5_n...,64.383688,rmse
...,...,...,...,...
5287,Material_81,XGBRegressor_learning_rate=0.1_max_depth=10_n_...,9489.025062,rmse
5288,Material_82,XGBRegressor_learning_rate=0.1_max_depth=10_n_...,13.258616,rmse
5289,Material_83,XGBRegressor_learning_rate=0.1_max_depth=10_n_...,8.545930,rmse
5290,Material_84,XGBRegressor_learning_rate=0.1_max_depth=10_n_...,10.198462,rmse


In [ ]:
def select_best_model(metrics_df):
    """
    Selects the best model for each unique_id based on the minimum metric value.

    Parameters:
        metrics_df (pd.DataFrame): A DataFrame containing unique_id, model, metric, and rmse columns.

    Returns:
        pd.DataFrame: A DataFrame with unique_id, the best model, and the corresponding metric value.
    """
    # Remove rows with NaN values in the 'rmse' column
    metrics_df = metrics_df.dropna(subset=['rmse'])

    # Convert 'unique_id' to string type to avoid category-related issues
    metrics_df['unique_id'] = metrics_df['unique_id'].astype(str)

    # Group by 'unique_id' and find the row with minimum 'rmse'
    best_models_df = metrics_df.loc[metrics_df.groupby('unique_id')['rmse'].idxmin()]

    return best_models_df[['unique_id', 'model', 'rmse']]



In [ ]:
best_models = select_best_model(evaluation_df)

In [ ]:
best_models

,unique_id,model,rmse
4452,Material_1,XGBRegressor_learning_rate=0.05_max_depth=10_n...,54.836482
3277,Material_10,XGBRegressor_learning_rate=0.01_max_depth=7_n_...,55.393952
674,Material_11,LGBMRegressor_learning_rate=0.05_max_depth=7_n...,46.025931
3615,Material_12,XGBRegressor_learning_rate=0.01_max_depth=10_n...,34.923250
4456,Material_13,XGBRegressor_learning_rate=0.05_max_depth=10_n...,50.950542
...,...,...,...
2515,Material_81,LGBMRegressor_learning_rate=0.1_max_depth=7_n_...,9123.196359
4112,Material_82,XGBRegressor_learning_rate=0.05_max_depth=7_n_...,9.749532
1509,Material_83,LGBMRegressor_learning_rate=0.05_max_depth=10_...,7.135424
2686,Material_84,LGBMRegressor_learning_rate=0.1_max_depth=10_n...,8.474572


In [ ]:
best_models.to_csv('best_models_ML.csv', index=False)